In [1]:
import time
import numpy as np
import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [2]:
!mkdir data
!wget -q -O data/ptb.zip https://ibm.box.com/shared/static/z2yvmhbskc45xd2a9a4kkn6hg4g4kj5r.zip
!unzip -o data/ptb.zip -d data
!cp data/ptb/reader.py .
import reader

Archive:  data/ptb.zip
   creating: data/ptb/
  inflating: data/ptb/reader.py      
   creating: data/__MACOSX/
   creating: data/__MACOSX/ptb/
  inflating: data/__MACOSX/ptb/._reader.py  
  inflating: data/__MACOSX/._ptb     


In [3]:
!wget http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz 
!tar xzf simple-examples.tgz -C data/

--2020-05-06 08:28:17--  http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz
Resolving www.fit.vutbr.cz (www.fit.vutbr.cz)... 147.229.9.23, 2001:67c:1220:809::93e5:917
Connecting to www.fit.vutbr.cz (www.fit.vutbr.cz)|147.229.9.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34869662 (33M) [application/x-gtar]
Saving to: ‘simple-examples.tgz’

simple-examples.tgz 100%[===================>]  33.25M  4.03MB/s    in 9.3s    

2020-05-06 08:28:27 (3.58 MB/s) - ‘simple-examples.tgz’ saved [34869662/34869662]



In [0]:
init_scale = 0.1
learning_rate = 1.0
max_grad_norm = 5
num_layers = 2
num_steps = 20
hidden_size_l1 = 256
hidden_size_l2 = 128
max_epoch_decay_lr = 4
max_epoch = 15
keep_prob = 1
decay = 0.5
batch_size = 60
vocab_size = 10000
embeding_vector_size = 200
is_training = 1
data_dir = "data/simple-examples/data/"

In [2]:
!pip install tensorflow==1.12.0

     |████████████████████████████████| 83.1MB 35kB/s 
     |████████████████████████████████| 3.1MB 34.8MB/s 
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1
  Found existing installation: tensorflow 2.2.0rc4
    Uninstalling tensorflow-2.2.0rc4:
      Successfully uninstalled tensorflow-2.2.0rc4


In [0]:
session = tf.InteractiveSession()

In [0]:
raw_data = reader.ptb_raw_data(data_dir)

In [0]:
train_data, valid_data, test_data, vocab, word_to_id = raw_data

In [8]:
len(train_data)

929589

In [9]:
print(train_data[0:100])

[9970, 9971, 9972, 9974, 9975, 9976, 9980, 9981, 9982, 9983, 9984, 9986, 9987, 9988, 9989, 9991, 9992, 9993, 9994, 9995, 9996, 9997, 9998, 9999, 2, 9256, 1, 3, 72, 393, 33, 2133, 0, 146, 19, 6, 9207, 276, 407, 3, 2, 23, 1, 13, 141, 4, 1, 5465, 0, 3081, 1596, 96, 2, 7682, 1, 3, 72, 393, 8, 337, 141, 4, 2477, 657, 2170, 955, 24, 521, 6, 9207, 276, 4, 39, 303, 438, 3684, 2, 6, 942, 4, 3150, 496, 263, 5, 138, 6092, 4241, 6036, 30, 988, 6, 241, 760, 4, 1015, 2786, 211, 6, 96, 4]


In [10]:
def id_to_word(id_list):
    line = []
    for w in id_list:
        for word, wid in word_to_id.items():
            if wid == w:
                line.append(word)
    return line            
                

print(id_to_word(train_data[0:100]))

['aer', 'banknote', 'berlitz', 'calloway', 'centrust', 'cluett', 'fromstein', 'gitano', 'guterman', 'hydro-quebec', 'ipo', 'kia', 'memotec', 'mlx', 'nahb', 'punts', 'rake', 'regatta', 'rubens', 'sim', 'snack-food', 'ssangyong', 'swapo', 'wachter', '<eos>', 'pierre', '<unk>', 'N', 'years', 'old', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'nov.', 'N', '<eos>', 'mr.', '<unk>', 'is', 'chairman', 'of', '<unk>', 'n.v.', 'the', 'dutch', 'publishing', 'group', '<eos>', 'rudolph', '<unk>', 'N', 'years', 'old', 'and', 'former', 'chairman', 'of', 'consolidated', 'gold', 'fields', 'plc', 'was', 'named', 'a', 'nonexecutive', 'director', 'of', 'this', 'british', 'industrial', 'conglomerate', '<eos>', 'a', 'form', 'of', 'asbestos', 'once', 'used', 'to', 'make', 'kent', 'cigarette', 'filters', 'has', 'caused', 'a', 'high', 'percentage', 'of', 'cancer', 'deaths', 'among', 'a', 'group', 'of']


In [0]:
itera = reader.ptb_iterator(train_data, batch_size, num_steps)
first_touple = itera.__next__()
x = first_touple[0]
y = first_touple[1]

In [12]:
print(x)

[[9970 9971 9972 ... 9993 9994 9995]
 [ 901   33 3361 ...  241   13 2420]
 [2654    6  334 ...  514    8  605]
 ...
 [7831   36 1678 ...    4 4558  157]
 [  59 2070 2433 ...  400    1 1173]
 [2097    3    2 ... 2043   23    1]]


In [13]:
x.shape

(60, 20)

In [0]:
_input_data = tf.placeholder(tf.int32 , [batch_size, num_steps])
_targets = tf.placeholder(tf.int32 , [batch_size, num_steps])

In [0]:
feed_dict = {_input_data:x, _targets:y}

In [17]:
session.run(_input_data, feed_dict)

array([[9970, 9971, 9972, ..., 9993, 9994, 9995],
       [ 901,   33, 3361, ...,  241,   13, 2420],
       [2654,    6,  334, ...,  514,    8,  605],
       ...,
       [7831,   36, 1678, ...,    4, 4558,  157],
       [  59, 2070, 2433, ...,  400,    1, 1173],
       [2097,    3,    2, ..., 2043,   23,    1]], dtype=int32)

In [18]:
lstm_cell_l1 = tf.contrib.rnn.BasicLSTMCell(hidden_size_l1, forget_bias=0.0)
lstm_cell_l2 = tf.contrib.rnn.BasicLSTMCell(hidden_size_l2, forget_bias=0.0)
stacked_lstm = tf.contrib.rnn.MultiRNNCell([lstm_cell_l1, lstm_cell_l2])

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


In [19]:
_initial_state = stacked_lstm.zero_state(batch_size, tf.float32)
_initial_state

(LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState/zeros:0' shape=(60, 256) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState/zeros_1:0' shape=(60, 256) dtype=float32>),
 LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState_1/zeros:0' shape=(60, 128) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState_1/zeros_1:0' shape=(60, 128) dtype=float32>))

In [21]:
session.run(_initial_state, feed_dict)

(LSTMStateTuple(c=array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), h=array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)),
 LSTMStateTuple(c=array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), h=array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ...,

In [0]:
embedding_vocab = tf.get_variable("embedding_vocab", [vocab_size, embeding_vector_size])  

In [23]:
session.run(tf.global_variables_initializer())
session.run(embedding_vocab)

array([[-7.1576815e-03, -2.3444863e-02,  2.1636765e-02, ...,
        -2.3458648e-02,  1.1177074e-02, -2.3858456e-02],
       [-1.8274082e-02,  1.1337671e-02, -1.0738964e-02, ...,
         5.1089767e-03,  3.0456204e-03,  6.9164876e-03],
       [-2.3984347e-02,  5.0297678e-03, -2.2850571e-02, ...,
        -1.9178528e-02,  1.2938876e-02,  2.2238284e-02],
       ...,
       [ 1.2332119e-02,  7.6370314e-05,  2.1143086e-02, ...,
         1.7066214e-02, -3.7131514e-03,  1.5743989e-02],
       [-9.2057111e-03, -2.0477977e-03,  1.2206256e-02, ...,
        -1.7645882e-02,  1.1226237e-02, -1.5508312e-02],
       [ 2.7251542e-03, -2.2815628e-02, -1.9703440e-02, ...,
         4.0978212e-03,  2.2724506e-02, -4.0931590e-03]], dtype=float32)

In [24]:
inputs = tf.nn.embedding_lookup(embedding_vocab, _input_data)  #shape=(30, 20, 200) 
inputs

<tf.Tensor 'embedding_lookup/Identity:0' shape=(60, 20, 200) dtype=float32>

In [26]:
session.run(inputs[0], feed_dict)

array([[ 0.00325009,  0.01081089,  0.0200482 , ...,  0.0022178 ,
         0.01401596, -0.00581577],
       [ 0.00442274,  0.01126799, -0.00109616, ..., -0.00493266,
         0.00903059,  0.00655018],
       [ 0.02124366,  0.02348461,  0.02135952, ...,  0.01759367,
         0.01497137, -0.01829221],
       ...,
       [-0.01907893,  0.01818139,  0.02366877, ...,  0.02151349,
        -0.0106791 , -0.00556704],
       [ 0.00410514,  0.00279766,  0.00547559, ..., -0.01219951,
        -0.02211702, -0.00371548],
       [-0.02399711,  0.02415726, -0.00398123, ..., -0.01793151,
         0.00649729, -0.00648275]], dtype=float32)

In [0]:
outputs, new_state =  tf.nn.dynamic_rnn(stacked_lstm, inputs, initial_state=_initial_state)

In [28]:
session.run(tf.global_variables_initializer())
session.run(outputs[0], feed_dict)

array([[ 1.00989251e-04,  6.92237154e-05,  1.73357417e-04, ...,
         4.05761442e-04, -2.25753283e-05,  9.50943941e-05],
       [ 3.96683870e-04,  4.82247618e-04, -1.18771946e-04, ...,
         5.07148216e-04,  2.40567402e-04,  4.75475361e-04],
       [ 7.95443717e-04,  3.01503838e-04, -2.29054000e-04, ...,
         7.91834085e-04,  3.37273232e-04,  8.74314515e-04],
       ...,
       [-6.63069077e-04,  8.78425606e-04,  8.69163632e-05, ...,
        -2.73895886e-04,  1.56508409e-04, -8.33585102e-04],
       [-1.12435629e-03,  6.06384070e-04,  3.30563111e-04, ...,
        -2.38942710e-04, -1.14292467e-04, -9.45392763e-04],
       [-6.93089038e-04,  1.38926538e-04,  1.05895603e-03, ...,
         3.86264612e-04,  2.89802731e-04, -6.68803346e-04]], dtype=float32)

In [30]:
output = tf.reshape(outputs, [-1, hidden_size_l2])
output

<tf.Tensor 'Reshape:0' shape=(1200, 128) dtype=float32>

In [0]:
softmax_w = tf.get_variable("softmax_w", [hidden_size_l2, vocab_size]) #[200x1000]
softmax_b = tf.get_variable("softmax_b", [vocab_size]) #[1x1000]
logits = tf.matmul(output, softmax_w) + softmax_b
prob = tf.nn.softmax(logits)

In [35]:
session.run(tf.global_variables_initializer())
output_words_prob = session.run(prob, feed_dict)
print("shape of the output: ", output_words_prob.shape)
print("The probability of observing words in t=0 to t=20", output_words_prob[0:20])

shape of the output:  (1200, 10000)
The probability of observing words in t=0 to t=20 [[1.01613296e-04 9.94720831e-05 1.00765646e-04 ... 1.00619422e-04
  9.99091353e-05 9.86637606e-05]
 [1.01628037e-04 9.94681322e-05 1.00769597e-04 ... 1.00622696e-04
  9.98977921e-05 9.86743762e-05]
 [1.01631325e-04 9.94678558e-05 1.00774392e-04 ... 1.00621925e-04
  9.98932446e-05 9.86833547e-05]
 ...
 [1.01605583e-04 9.94582515e-05 1.00770769e-04 ... 1.00610567e-04
  9.99066324e-05 9.86663072e-05]
 [1.01606274e-04 9.94520960e-05 1.00770529e-04 ... 1.00608457e-04
  9.99003241e-05 9.86626983e-05]
 [1.01607540e-04 9.94602306e-05 1.00769787e-04 ... 1.00601661e-04
  9.98939940e-05 9.86595041e-05]]


In [36]:
np.argmax(output_words_prob[0:20], axis=1)

array([ 343,  343,  343,  343, 9540, 9540, 6745, 8953, 8953, 9995, 9995,
       3928, 3928, 3928, 6755, 6755, 9923, 9923, 3928, 3928])

In [37]:
y[0]

array([9971, 9972, 9974, 9975, 9976, 9980, 9981, 9982, 9983, 9984, 9986,
       9987, 9988, 9989, 9991, 9992, 9993, 9994, 9995, 9996], dtype=int32)

In [39]:
targ = session.run(_targets, feed_dict) 
targ[0]

array([9971, 9972, 9974, 9975, 9976, 9980, 9981, 9982, 9983, 9984, 9986,
       9987, 9988, 9989, 9991, 9992, 9993, 9994, 9995, 9996], dtype=int32)

In [0]:
loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], [tf.reshape(_targets, [-1])],[tf.ones([batch_size * num_steps])])

In [41]:
session.run(loss, feed_dict)[:10]

array([9.204273, 9.224012, 9.216199, 9.221151, 9.195596, 9.204564,
       9.216711, 9.209148, 9.20055 , 9.223702], dtype=float32)

In [42]:
cost = tf.reduce_sum(loss) / batch_size
session.run(tf.global_variables_initializer())
session.run(cost, feed_dict)

184.19994

In [0]:
lr = tf.Variable(0.0, trainable=False)
# Create the gradient descent optimizer with our learning rate
optimizer = tf.train.GradientDescentOptimizer(lr)

In [44]:
tvars = tf.trainable_variables()
tvars

[<tf.Variable 'embedding_vocab:0' shape=(10000, 200) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0' shape=(456, 1024) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias:0' shape=(1024,) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/kernel:0' shape=(384, 512) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/bias:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'softmax_w:0' shape=(128, 10000) dtype=float32_ref>,
 <tf.Variable 'softmax_b:0' shape=(10000,) dtype=float32_ref>]

In [45]:
[v.name for v in tvars]

['embedding_vocab:0',
 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0',
 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias:0',
 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/kernel:0',
 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/bias:0',
 'softmax_w:0',
 'softmax_b:0']

In [47]:
var_x = tf.placeholder(tf.float32)
var_y = tf.placeholder(tf.float32) 
func_test = 2.0 * var_x * var_x + 3.0 * var_x * var_y
session.run(tf.global_variables_initializer())
session.run(func_test, {var_x:1.0,var_y:2.0})

8.0

In [49]:
var_grad = tf.gradients(func_test, [var_x])
session.run(var_grad, {var_x:1.0,var_y:2.0})

[10.0]

In [51]:
var_grad = tf.gradients(func_test, [var_y])
session.run(var_grad, {var_x:1.0, var_y:2.0})

[3.0]

In [52]:
tf.gradients(cost, tvars)

 <tf.Tensor 'gradients_4/rnn/while/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/MatMul/Enter_grad/b_acc_3:0' shape=(456, 1024) dtype=float32>,
 <tf.Tensor 'gradients_4/rnn/while/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/BiasAdd/Enter_grad/b_acc_3:0' shape=(1024,) dtype=float32>,
 <tf.Tensor 'gradients_4/rnn/while/rnn/multi_rnn_cell/cell_1/basic_lstm_cell/MatMul/Enter_grad/b_acc_3:0' shape=(384, 512) dtype=float32>,
 <tf.Tensor 'gradients_4/rnn/while/rnn/multi_rnn_cell/cell_1/basic_lstm_cell/BiasAdd/Enter_grad/b_acc_3:0' shape=(512,) dtype=float32>,
 <tf.Tensor 'gradients_4/MatMul_grad/MatMul_1:0' shape=(128, 10000) dtype=float32>,
 <tf.Tensor 'gradients_4/add_grad/Reshape_1:0' shape=(10000,) dtype=float32>]

In [54]:
grad_t_list = tf.gradients(cost, tvars)
session.run(grad_t_list,feed_dict)

[IndexedSlicesValue(values=array([[-2.7893816e-07, -1.2814845e-05, -1.3136873e-05, ...,
        -7.3561187e-06, -4.3364307e-06, -1.0902732e-05],
       [ 1.6519586e-07, -1.7336493e-05, -1.3297089e-05, ...,
        -1.1260923e-05, -3.6407571e-06, -7.2892703e-06],
       [ 1.3327696e-06,  2.7600042e-06, -4.5080787e-06, ...,
        -5.6128933e-06, -5.8137848e-06, -5.4516358e-06],
       ...,
       [-5.8048096e-07,  5.8552064e-06,  5.5862770e-06, ...,
         1.0751618e-06, -9.3951442e-08,  1.0573129e-05],
       [-8.6072878e-06,  5.7813136e-06,  3.8972007e-06, ...,
        -4.3460022e-06, -1.1080484e-06,  9.0330959e-06],
       [-9.8562978e-06,  9.1688526e-06,  1.5621805e-06, ...,
        -1.9780257e-06,  6.0257344e-06,  1.1340519e-05]], dtype=float32), indices=array([9970, 9971, 9972, ..., 2043,   23,    1], dtype=int32), dense_shape=array([10000,   200], dtype=int32)),
 array([[-5.60766473e-08,  3.01972598e-08, -4.73208672e-09, ...,
          2.93886622e-08, -1.85395344e-08,  4.80393

In [57]:
grads, _ = tf.clip_by_global_norm(grad_t_list, max_grad_norm)
grads

 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_1:0' shape=(456, 1024) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_2:0' shape=(1024,) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_3:0' shape=(384, 512) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_4:0' shape=(512,) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_5:0' shape=(128, 10000) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_6:0' shape=(10000,) dtype=float32>]

In [58]:
session.run(grads, feed_dict)

[IndexedSlicesValue(values=array([[-2.7893816e-07, -1.2814845e-05, -1.3136873e-05, ...,
        -7.3561187e-06, -4.3364307e-06, -1.0902732e-05],
       [ 1.6519586e-07, -1.7336493e-05, -1.3297089e-05, ...,
        -1.1260923e-05, -3.6407571e-06, -7.2892703e-06],
       [ 1.3327696e-06,  2.7600042e-06, -4.5080787e-06, ...,
        -5.6128933e-06, -5.8137848e-06, -5.4516358e-06],
       ...,
       [-5.8048096e-07,  5.8552064e-06,  5.5862770e-06, ...,
         1.0751618e-06, -9.3951442e-08,  1.0573129e-05],
       [-8.6072878e-06,  5.7813136e-06,  3.8972007e-06, ...,
        -4.3460022e-06, -1.1080484e-06,  9.0330959e-06],
       [-9.8562978e-06,  9.1688526e-06,  1.5621805e-06, ...,
        -1.9780257e-06,  6.0257344e-06,  1.1340519e-05]], dtype=float32), indices=array([9970, 9971, 9972, ..., 2043,   23,    1], dtype=int32), dense_shape=array([10000,   200], dtype=int32)),
 array([[-5.60766473e-08,  3.01972598e-08, -4.73208672e-09, ...,
          2.93886622e-08, -1.85395344e-08,  4.80393

In [0]:
train_op = optimizer.apply_gradients(zip(grads, tvars))

In [0]:
session.run(tf.global_variables_initializer())
session.run(train_op, feed_dict)

In [61]:
hidden_size_l1

256

In [0]:
class PTBModel(object):

    def __init__(self, action_type):
        ######################################
        # Setting parameters for ease of use #
        ######################################
        self.batch_size = batch_size
        self.num_steps = num_steps
        self.hidden_size_l1 = hidden_size_l1
        self.hidden_size_l2 = hidden_size_l2
        self.vocab_size = vocab_size
        self.embeding_vector_size = embeding_vector_size
        ###############################################################################
        # Creating placeholders for our input data and expected outputs (target data) #
        ###############################################################################
        self._input_data = tf.placeholder(tf.int32, [batch_size, num_steps]) #[30#20]
        self._targets = tf.placeholder(tf.int32, [batch_size, num_steps]) #[30#20]

        ##########################################################################
        # Creating the LSTM cell structure and connect it with the RNN structure #
        ##########################################################################
        # Create the LSTM unit. 
        # This creates only the structure for the LSTM and has to be associated with a RNN unit still.
        # The argument n_hidden(size=200) of BasicLSTMCell is size of hidden layer, that is, the number of hidden units of the LSTM (inside A).
        # Size is the same as the size of our hidden layer, and no bias is added to the Forget Gate. 
        # LSTM cell processes one word at a time and computes probabilities of the possible continuations of the sentence.
        lstm_cell_l1 = tf.contrib.rnn.BasicLSTMCell(self.hidden_size_l1, forget_bias=0.0)
        lstm_cell_l2 = tf.contrib.rnn.BasicLSTMCell(self.hidden_size_l2, forget_bias=0.0)
        
        # Unless you changed keep_prob, this won't actually execute -- this is a dropout wrapper for our LSTM unit
        # This is an optimization of the LSTM output, but is not needed at all
        if action_type == "is_training" and keep_prob < 1:
            lstm_cell_l1 = tf.contrib.rnn.DropoutWrapper(lstm_cell_l1, output_keep_prob=keep_prob)
            lstm_cell_l2 = tf.contrib.rnn.DropoutWrapper(lstm_cell_l2, output_keep_prob=keep_prob)
        
        # By taking in the LSTM cells as parameters, the MultiRNNCell function junctions the LSTM units to the RNN units.
        # RNN cell composed sequentially of multiple simple cells.
        stacked_lstm = tf.contrib.rnn.MultiRNNCell([lstm_cell_l1, lstm_cell_l2])

        # Define the initial state, i.e., the model state for the very first data point
        # It initialize the state of the LSTM memory. The memory state of the network is initialized with a vector of zeros and gets updated after reading each word.
        self._initial_state = stacked_lstm.zero_state(batch_size, tf.float32)

        ####################################################################
        # Creating the word embeddings and pointing them to the input data #
        ####################################################################
        with tf.device("/cpu:0"):
            # Create the embeddings for our input data. Size is hidden size.
            embedding = tf.get_variable("embedding", [vocab_size, self.embeding_vector_size])  #[10000x200]
            # Define where to get the data for our embeddings from
            inputs = tf.nn.embedding_lookup(embedding, self._input_data)

        # Unless you changed keep_prob, this won't actually execute -- this is a dropout addition for our inputs
        # This is an optimization of the input processing and is not needed at all
        if action_type == "is_training" and keep_prob < 1:
            inputs = tf.nn.dropout(inputs, keep_prob)

        ############################################
        # Creating the input structure for our RNN #
        ############################################
        # Input structure is 20x[30x200]
        # Considering each word is represended by a 200 dimentional vector, and we have 30 batchs, we create 30 word-vectors of size [30xx2000]
        # inputs = [tf.squeeze(input_, [1]) for input_ in tf.split(1, num_steps, inputs)]
        # The input structure is fed from the embeddings, which are filled in by the input data
        # Feeding a batch of b sentences to a RNN:
        # In step 1,  first word of each of the b sentences (in a batch) is input in parallel.  
        # In step 2,  second word of each of the b sentences is input in parallel. 
        # The parallelism is only for efficiency.  
        # Each sentence in a batch is handled in parallel, but the network sees one word of a sentence at a time and does the computations accordingly. 
        # All the computations involving the words of all sentences in a batch at a given time step are done in parallel. 

        ####################################################################################################
        # Instantiating our RNN model and retrieving the structure for returning the outputs and the state #
        ####################################################################################################
        
        outputs, state = tf.nn.dynamic_rnn(stacked_lstm, inputs, initial_state=self._initial_state)
        #########################################################################
        # Creating a logistic unit to return the probability of the output word #
        #########################################################################
        output = tf.reshape(outputs, [-1, self.hidden_size_l2])
        softmax_w = tf.get_variable("softmax_w", [self.hidden_size_l2, vocab_size]) #[200x1000]
        softmax_b = tf.get_variable("softmax_b", [vocab_size]) #[1x1000]
        logits = tf.matmul(output, softmax_w) + softmax_b
        logits = tf.reshape(logits, [self.batch_size, self.num_steps, vocab_size])
        prob = tf.nn.softmax(logits)
        out_words = tf.argmax(prob, axis=2)
        self._output_words = out_words
        #########################################################################
        # Defining the loss and cost functions for the model's learning to work #
        #########################################################################
            

        # Use the contrib sequence loss and average over the batches
        loss = tf.contrib.seq2seq.sequence_loss(
            logits,
            self.targets,
            tf.ones([batch_size, num_steps], dtype=tf.float32),
            average_across_timesteps=False,
            average_across_batch=True)
    
#         loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], [tf.reshape(self._targets, [-1])],
#                                                       [tf.ones([batch_size * num_steps])])
        self._cost = tf.reduce_sum(loss)

        # Store the final state
        self._final_state = state

        #Everything after this point is relevant only for training
        if action_type != "is_training":
            return

        #################################################
        # Creating the Training Operation for our Model #
        #################################################
        # Create a variable for the learning rate
        self._lr = tf.Variable(0.0, trainable=False)
        # Get all TensorFlow variables marked as "trainable" (i.e. all of them except _lr, which we just created)
        tvars = tf.trainable_variables()
        # Define the gradient clipping threshold
        grads, _ = tf.clip_by_global_norm(tf.gradients(self._cost, tvars), max_grad_norm)
        # Create the gradient descent optimizer with our learning rate
        optimizer = tf.train.GradientDescentOptimizer(self.lr)
        # Create the training TensorFlow Operation through our optimizer
        self._train_op = optimizer.apply_gradients(zip(grads, tvars))

    # Helper functions for our LSTM RNN class

    # Assign the learning rate for this model
    def assign_lr(self, session, lr_value):
        session.run(tf.assign(self.lr, lr_value))

    # Returns the input data for this model at a point in time
    @property
    def input_data(self):
        return self._input_data


    
    # Returns the targets for this model at a point in time
    @property
    def targets(self):
        return self._targets

    # Returns the initial state for this model
    @property
    def initial_state(self):
        return self._initial_state

    # Returns the defined Cost
    @property
    def cost(self):
        return self._cost

    # Returns the final state for this model
    @property
    def final_state(self):
        return self._final_state
    
    # Returns the final output words for this model
    @property
    def final_output_words(self):
        return self._output_words
    
    # Returns the current learning rate for this model
    @property
    def lr(self):
        return self._lr

    # Returns the training operation defined for this model
    @property
    def train_op(self):
        return self._train_op

In [0]:
def run_one_epoch(session, m, data, eval_op, verbose=False):

    #Define the epoch size based on the length of the data, batch size and the number of steps
    epoch_size = ((len(data) // m.batch_size) - 1) // m.num_steps
    start_time = time.time()
    costs = 0.0
    iters = 0

    state = session.run(m.initial_state)
    
    #For each step and data point
    for step, (x, y) in enumerate(reader.ptb_iterator(data, m.batch_size, m.num_steps)):
        
        #Evaluate and return cost, state by running cost, final_state and the function passed as parameter
        cost, state, out_words, _ = session.run([m.cost, m.final_state, m.final_output_words, eval_op],
                                     {m.input_data: x,
                                      m.targets: y,
                                      m.initial_state: state})

        #Add returned cost to costs (which keeps track of the total costs for this epoch)
        costs += cost
        
        #Add number of steps to iteration counter
        iters += m.num_steps

        if verbose and step % (epoch_size // 10) == 10:
            print("Itr %d of %d, perplexity: %.3f speed: %.0f wps" % (step , epoch_size, np.exp(costs / iters), iters * m.batch_size / (time.time() - start_time)))

    # Returns the Perplexity rating for us to keep track of how the model is evolving
    return np.exp(costs / iters)

In [0]:
raw_data = reader.ptb_raw_data(data_dir)
train_data, valid_data, test_data, _, _ = raw_data

In [0]:
with tf.Graph().as_default(), tf.Session() as session:
    initializer = tf.random_uniform_initializer(-init_scale, init_scale)
    
    # Instantiates the model for training
    # tf.variable_scope add a prefix to the variables created with tf.get_variable
    with tf.variable_scope("model", reuse=None, initializer=initializer):
        m = PTBModel("is_training")
        
    # Reuses the trained parameters for the validation and testing models
    # They are different instances but use the same variables for weights and biases, they just don't change when data is input
    with tf.variable_scope("model", reuse=True, initializer=initializer):
        mvalid = PTBModel("is_validating")
        mtest = PTBModel("is_testing")

    #Initialize all variables
    tf.global_variables_initializer().run()

    for i in range(max_epoch):
        # Define the decay for this epoch
        lr_decay = decay ** max(i - max_epoch_decay_lr, 0.0)
        
        # Set the decayed learning rate as the learning rate for this epoch
        m.assign_lr(session, learning_rate * lr_decay)

        print("Epoch %d : Learning rate: %.3f" % (i + 1, session.run(m.lr)))
        
        # Run the loop for this epoch in the training model
        train_perplexity = run_one_epoch(session, m, train_data, m.train_op, verbose=True)
        print("Epoch %d : Train Perplexity: %.3f" % (i + 1, train_perplexity))
        
        # Run the loop for this epoch in the validation model
        valid_perplexity = run_one_epoch(session, mvalid, valid_data, tf.no_op())
        print("Epoch %d : Valid Perplexity: %.3f" % (i + 1, valid_perplexity))
    
    # Run the loop in the testing model to see how effective was our training
    test_perplexity = run_one_epoch(session, mtest, test_data, tf.no_op())
    
    print("Test Perplexity: %.3f" % test_perplexity)

Epoch 1 : Learning rate: 1.000
Itr 10 of 774, perplexity: 3650.990 speed: 1645 wps
Itr 87 of 774, perplexity: 1248.877 speed: 1682 wps
Itr 164 of 774, perplexity: 966.082 speed: 1682 wps
Itr 241 of 774, perplexity: 807.103 speed: 1681 wps
Itr 318 of 774, perplexity: 714.737 speed: 1683 wps
Itr 395 of 774, perplexity: 640.670 speed: 1685 wps
Itr 472 of 774, perplexity: 582.377 speed: 1690 wps
Itr 549 of 774, perplexity: 529.034 speed: 1690 wps
Itr 626 of 774, perplexity: 487.013 speed: 1685 wps
Itr 703 of 774, perplexity: 453.350 speed: 1682 wps
Epoch 1 : Train Perplexity: 429.255
Epoch 1 : Valid Perplexity: 266.269
Epoch 2 : Learning rate: 1.000
Itr 10 of 774, perplexity: 278.182 speed: 1664 wps
Itr 87 of 774, perplexity: 239.337 speed: 1677 wps
Itr 164 of 774, perplexity: 229.089 speed: 1688 wps
Itr 241 of 774, perplexity: 218.988 speed: 1692 wps
Itr 318 of 774, perplexity: 216.391 speed: 1695 wps
Itr 395 of 774, perplexity: 210.586 speed: 1698 wps
Itr 472 of 774, perplexity: 206.363 